In [1]:
from fastai.vision.all import *
import params
import wandb

In [2]:
URL = 'file:///Users/jobdulo/Downloads/earth_sem_seg.zip'

In [3]:
path = untar_data(URL, force_download=False)

In [4]:
path.ls()

(#9) [Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 8'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 6'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/classes.json'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 7'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 5'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 2'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 3'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 4')]

In [5]:
(path/'Tile 1').ls()

(#2) [Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/images'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/masks')]

In [6]:
(path/'Tile 1/images').ls()

(#9) [Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/images/image_part_004.jpg'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/images/image_part_005.jpg'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/images/image_part_007.jpg'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/images/image_part_006.jpg'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/images/image_part_002.jpg'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/images/image_part_003.jpg'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/images/image_part_001.jpg'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/images/image_part_008.jpg'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/images/image_part_009.jpg')]

In [7]:
(path/'Tile 1/masks').ls()

(#9) [Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/masks/image_part_004.png'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/masks/image_part_005.png'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/masks/image_part_007.png'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/masks/image_part_006.png'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/masks/image_part_002.png'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/masks/image_part_003.png'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/masks/image_part_001.png'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/masks/image_part_008.png'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/masks/image_part_009.png')]

In [40]:
def label_func(fname):
    return (fname.parent.parent/"masks")/f"{fname.stem}.png"

def get_classes_per_image(mask_data, class_labels):
    for i in mask_data:
        print(i)
#     unique = list(np.unique(mask_data))
    print("unique: ", unique)
    result_dict = {}
    print(class_labels)
    for _class in class_labels.keys():
        result_dict[class_labels[_class]] = int(_class in unique)
    print("result_dict: ", result_dict)
    return result_dict

def _create_table(image_files, class_labels):
    # create a table with the datasets
    labels = [str(class_labels[_lab]) for _lab in list(class_labels)]
    table = wandb.Table(columns=["File_Name", "Images", "Dataset"] + labels)
    
    for i, image_file in progress_bar(enumerate(image_files), total=len(image_files)):
        image = Image.open(image_file)
        mask_data = np.array(Image.open(label_func(image_file)))
        print("old mask data: ", mask_data)
        # reshape mask_data to 2D
        mask_data = np.reshape(mask_data, (mask_data.shape[0], mask_data.shape[1]*mask_data.shape[2]))
        print("new mask data: ", mask_data)
        class_in_image = get_classes_per_image(mask_data, class_labels)
        table.add_data(
            str(image_file.name),
            wandb.Image(
                    image,
                    masks={
                        "predictions": {
                            "mask_data": mask_data,
                            "class_labels": class_labels,
                        }
                    }
            ),
            "big_earth_net", # we don't have a dataset split yet
            *[class_in_image[_lab] for _lab in labels]
        )
    
    return table

In [9]:
!wandb login

wandb: Currently logged in as: dulo. Use `wandb login --relogin` to force relogin


In [10]:
import be_params

In [11]:
run = wandb.init(project=be_params.WANDB_PROJECT, entity=be_params.ENTITY, job_type="data_upload")
raw_data_at = wandb.Artifact(params.RAW_DATA_AT, type="raw_data")

wandb: Currently logged in as: dulo. Use `wandb login --relogin` to force relogin


### add the images and the label masks

In [12]:
raw_data_at.add_dir(path/'Tile 1/images', name='images')
raw_data_at.add_dir(path/'Tile 1/masks', name='masks')

wandb: Adding directory to artifact (/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/images)... Done. 0.1s
wandb: Adding directory to artifact (/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/masks)... Done. 0.1s


In [13]:
DEBUG = False # set this flag to True to use a small subset of data for testing

image_files = get_image_files(path/"Tile 1/images", recurse=False)

# sample a subset if DEBUG
if DEBUG: image_files = image_files[:10]

In [14]:
image_files

(#9) [Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/images/image_part_004.jpg'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/images/image_part_005.jpg'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/images/image_part_007.jpg'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/images/image_part_006.jpg'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/images/image_part_002.jpg'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/images/image_part_003.jpg'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/images/image_part_001.jpg'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/images/image_part_008.jpg'),Path('/Users/jobdulo/.fastai/data/earth_sem_seg/Tile 1/images/image_part_009.jpg')]

In [15]:
# classes = {"classes": [{"title": "Water", "shape": "polygon", "color": "#50E3C2", "geometry_config": {}}, {"title": "Land (unpaved area)", "shape": "polygon", "color": "#F5A623", "geometry_config": {}}, {"title": "Road", "shape": "polygon", "color": "#DE597F", "geometry_config": {}}, {"title": "Building", "shape": "polygon", "color": "#D0021B", "geometry_config": {}}, {"title": "Vegetation", "shape": "polygon", "color": "#417505", "geometry_config": {}}, {"title": "Unlabeled", "shape": "polygon", "color": "#9B9B9B", "geometry_config": {}}], "tags": []}

In [16]:
# class_dict = {item['color']:item['title'] for item in classes["classes"]}
# class_dict

In [41]:
# create a W& B table
table = _create_table(image_files, be_params.BE_CLASSES)

old mask data:  [[[132  41 246]
  [132  41 246]
  [132  41 246]
  ...
  [132  41 246]
  [132  41 246]
  [132  41 246]]

 [[132  41 246]
  [132  41 246]
  [132  41 246]
  ...
  [132  41 246]
  [132  41 246]
  [132  41 246]]

 [[132  41 246]
  [132  41 246]
  [132  41 246]
  ...
  [132  41 246]
  [132  41 246]
  [132  41 246]]

 ...

 [[132  41 246]
  [132  41 246]
  [132  41 246]
  ...
  [132  41 246]
  [132  41 246]
  [132  41 246]]

 [[132  41 246]
  [132  41 246]
  [132  41 246]
  ...
  [132  41 246]
  [132  41 246]
  [132  41 246]]

 [[132  41 246]
  [132  41 246]
  [132  41 246]
  ...
  [132  41 246]
  [132  41 246]
  [132  41 246]]]
new mask data:  [[132  41 246 ... 132  41 246]
 [132  41 246 ... 132  41 246]
 [132  41 246 ... 132  41 246]
 ...
 [132  41 246 ... 132  41 246]
 [132  41 246 ... 132  41 246]
 [132  41 246 ... 132  41 246]]
items:  2391
items:  2391
items:  2391
items:  2391
items:  2391
items:  2391
items:  2391
items:  2391
items:  2391
items:  2391
items:  2391
ite

NameError: name 'unique' is not defined

In [17]:
# for i, image_file in progress_bar(enumerate(image_files), total=len(image_files)):
#         image = Image.open(image_file)
#         mask_data = np.array(Image.open(label_func(image_file)))

In [18]:
# mask_data

In [19]:
# mask_data.ndim

In [20]:
raw_data_at.add(table, "big_earth_eda_table")

<ManifestEntry digest: dNVZ78Cs9GxQJU1roAYULg==>

In [21]:
run.log_artifact(raw_data_at)
run.finish()

In [29]:
maskx = [[132,  41, 246, 132,  41, 246], [132,  41, 246, 132,  41, 246]]

In [30]:
for i in maskx:
    print(i)

[132, 41, 246, 132, 41, 246]
[132, 41, 246, 132, 41, 246]
